# Boston housing

## Dane

In [40]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from sklearn.datasets import load_boston
warnings.filterwarnings('ignore')

In [41]:
boston_dataset = load_boston()
data = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)

In [42]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


Trzeba dodać zmienną celu

In [43]:
data['MEDV'] = boston_dataset.target

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  MEDV     506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [45]:
data.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


## Przygotowanie danych

Nie trzeba encodować można co najwyżej znormalizować

In [117]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(['MEDV'], axis=1), data['MEDV'], test_size=0.2)

In [113]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [114]:
#for col in X_train.columns:
#    scaler.fit(X_train[[col]])
#    X_train[col] = pd.DataFrame(scaler.transform(X_train[[col]]))
#    X_test[col] = pd.DataFrame(scaler.transform(X_test[[col]]))
# Nie jestem pewien dlaczego, ale min_max_scaler w tym przypadku generuje NaNy

## Model i uczenie bez strojenia

In [119]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

In [120]:
clf = SVR()
clf.fit(X_train, y_train)
mean_squared_error(y_test, clf.predict(X_test))

105.28565159703409

Również dość słaby wynik

## Strojenie random search 

In [121]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error

In [122]:
kernel=['rbf'] # jądro gaussowskie
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
gamma = [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5, 0.6, 0.9, 1]
param_grid = dict(kernel=kernel, C = C, gamma = gamma)

In [123]:
scorer = make_scorer(mean_squared_error, greater_is_better=False)
random = RandomizedSearchCV(estimator=SVR(epsilon = 0.01), param_distributions=param_grid, cv = 4, n_jobs=-1)

In [126]:
random_result = random.fit(X_train, y_train)

In [127]:
print("Wyniki na zbiorze treningowym:")
means = random_result.cv_results_['mean_test_score']
stds = random_result.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, random_result.cv_results_['params']):
    print("%0.3f (+/-%0.03f) dla %r"% (mean, std * 2, params))

Wyniki na zbiorze treningowym:
-0.028 (+/-0.045) dla {'kernel': 'rbf', 'gamma': 0.6, 'C': 0.001}
0.586 (+/-0.146) dla {'kernel': 'rbf', 'gamma': 0.0001, 'C': 100}
-0.028 (+/-0.045) dla {'kernel': 'rbf', 'gamma': 0.9, 'C': 0.001}
0.561 (+/-0.126) dla {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
-0.002 (+/-0.008) dla {'kernel': 'rbf', 'gamma': 0.5, 'C': 10000}
-0.025 (+/-0.044) dla {'kernel': 'rbf', 'gamma': 0.2, 'C': 1}
0.388 (+/-0.144) dla {'kernel': 'rbf', 'gamma': 0.01, 'C': 10}
0.397 (+/-0.145) dla {'kernel': 'rbf', 'gamma': 0.01, 'C': 100}
-0.028 (+/-0.044) dla {'kernel': 'rbf', 'gamma': 0.9, 'C': 1}
-0.028 (+/-0.044) dla {'kernel': 'rbf', 'gamma': 0.6, 'C': 1}


Najlepszy wynik dla **gamma : 0.5 C:10000**

## Dostosowanie modelu

In [129]:
clf = SVR(kernel = 'rbf', gamma = 0.9, C = 10000)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.010054003057696104

Troszkę gorszy wynik niż w poprzednim jednak zbiór jest dwa razy mniejszy.